# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard


In [2]:
import sqlite3
import nltk
import random
import numpy as np
import re
from collections import Counter, defaultdict

from string import punctuation

# Feel free to include your text patterns functions
# from text_functions_solutions import clean_tokenize, get_patterns
from functools import partial
from pathlib import Path
from nltk import sent_tokenize
from nltk.corpus import stopwords

### Text Preprocessing Pipeline Setup

The following are the helper functions for the text preprocessing pipeline. The pipeline includes the following transformations:

1. **Tokenize on whitespace** – split sentences into tokens based on spaces.
2. **Remove punctuation** – strip out punctuation marks from tokens.
3. **Keep only alphabetic tokens** – discard numbers, symbols, and mixed tokens.
4. **Remove stopwords** – filter out common words (e.g., "the", "and") that carry little meaning.
5. **Lowercase conversion** – normalize all words to lowercase.
6. **Join back to string** – reconstruct the cleaned tokens into a single string.

The `run_pipeline()` function executes these steps in sequence for any given sentence.


In [3]:
# set data location
data_location = Path("../datasets")

# constants
PUNCT_SET = set(punctuation)
TW_PUNCT_SET = PUNCT_SET - {"#"}
SW_ENG = stopwords.words("english")
WHITESPACE_RE = re.compile(r"\s+")
TWEET_RE = re.compile("(^b['\"])(.*)")


# helper functions
def tokenize_on_ws(text):
    return WHITESPACE_RE.split(text)


def remove_punct(tokens, punct_set=PUNCT_SET):
    cleaned = []
    for token in tokens:
        tok = "".join([ch for ch in token if ch not in punct_set])
        if tok:
            cleaned.append(tok)
    return cleaned


def is_alpha(tokens):
    return [token for token in tokens if token.isalpha()]


def remove_stopwords(tokens, sw=SW_ENG):
    tokens = [token for token in tokens if token.lower() not in sw]
    return tokens


def lowercase(tokens):
    return [token.lower() for token in tokens]


def join_to_string(tokens):
    return " ".join(tokens)


def clean_tweet(text):
    return TWEET_RE.match(text).group(2)


def run_pipeline(text, pipeline):
    tokens = str(text)

    for transform in pipeline:
        tokens = transform(tokens)

    return tokens

In [4]:
convention_db = sqlite3.connect(data_location / "2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text
for each party and prepare it for use in Naive Bayes.


In [5]:
convention_data = []

# fill the above list up with items that are themselves lists. The
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party.

query_results = convention_cur.execute(
    # -- your query here, pull only 2020 data.
    # -- Remove the party "Other".
    """    
    SELECT text, party 
    FROM conventions 
    WHERE party != 'Other';
    """
)

for row in query_results:
    # store the results in convention_data
    convention_data.append([row[0], row[1]])

In [6]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

In [7]:
party_counts = Counter(row[1] for row in convention_data)
print(party_counts)

total = party_counts[0] + party_counts[1]

Counter({'Democratic': 1551, 'Republican': 990})


Let's look at some random entries and see if they look right.


In [8]:
random.choices(convention_data, k=5)

[['Illinois.', 'Democratic'],
 ['This was people’s moms, dads, brothers, sisters, sons, daughters, all dying alone. And we know there’s a round two coming and mentally, physically, emotionally, I don’t know how many of us on the front lines are going to survive that second round.',
  'Democratic'],
 ['He would treat the conductor the same as he would the President of the United States. That is what dad taught us, that everybody deserves to be treated with dignity and respect.',
  'Democratic'],
 ['Insulin and other drug prices would have continued to rise while the record number of generic drugs would still be stuck in the pipeline. There’d be no price transparency. We wouldn’t have health plans up to 60% cheaper than Obamacare and we’d still be stuck with that infamous individual mandate. God forbid what the next four years would look like for in Joe Biden’s America, China would control our drug production. We’d be one step closer to government run healthcare. We wouldn’t just be unab

It'll be useful for us to have a large sample size than 2020 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html).


In [9]:
conv_sent_data = []

nltk.download("punkt")
nltk.download("punkt_tab")
for speech, party in convention_data:
    sentences = sent_tokenize(speech)
    conv_sent_data.append([sentences[0], party])

[nltk_data] Downloading package punkt to /home/junc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/junc/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Again, let's look at some random entries.


In [10]:
random.choices(conv_sent_data, k=5)

[['We came to America before I was one year old.', 'Democratic'],
 ['The differences between Trump-Pence and Biden- Harris are crystal clear.',
  'Republican'],
 ['The violence is rampant.', 'Republican'],
 ['All right.', 'Democratic'],
 ['Joe’s purpose has always driven him forward.', 'Democratic']]

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps:

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [11]:
speech_pipeline = [
    tokenize_on_ws,
    remove_punct,
    is_alpha,
    remove_stopwords,
    lowercase,
    join_to_string,
]

clean_conv_sent_data = []  # list of tuples (sentence, party), with sentence cleaned

for idx, sent_party in enumerate(conv_sent_data):
    # pass  # your code here
    # print(idx, sent_party[0], "\n")
    cleaned = run_pipeline(sent_party[0], speech_pipeline)
    # print(idx, cleaned)
    if cleaned:
        clean_conv_sent_data.append((cleaned, sent_party[1]))

random.choices(clean_conv_sent_data, k=5)

[('knows saving planet', 'Democratic'),
 ('oh well pretty well well expected', 'Democratic'),
 ('think day saw joe clearest one year anniversary boston marathon bombing',
  'Democratic'),
 ('style legislating leadership find much anymore', 'Democratic'),
 ('honor company great great champions', 'Republican')]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5.


In [12]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items():
    if count > word_cutoff:
        feature_words.add(word)

print(
    f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model."
)

With a word cutoff of 5, we have 628 as features in the model.


In [13]:
def conv_features(text, fw):
    """Given some text, this returns a dictionary holding the
    feature words.

    Args:
         * text: a piece of text in a continuous string. Assumes
         text has been cleaned and case folded.
         * fw: the *feature words* that we're considering. A word
         in `text` must be in fw in order to be returned. This
         prevents us from considering very rarely occurring words.

    Returns:
         A dictionary with the words in `text` that appear in `fw`.
         Words are only counted once.
         If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
         then this would return a dictionary of
         {'quick' : True,
          'fox' :    True}

    """

    # Your code here
    ret_dict = dict()
    words = text.split()
    for word in words:
        if word in fw:
            ret_dict[word] = True

    return ret_dict

In [14]:
assert len(feature_words) > 0
assert conv_features("obama was the president", feature_words) == {
    "obama": True,
    "president": True,
}
assert conv_features("some people in america are citizens", feature_words) == {
    "people": True,
    "america": True,
    "citizens": True,
}

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory.


In [15]:
featuresets = [
    (conv_features(text, feature_words), party) for (text, party) in convention_data
]

In [16]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [17]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.494


In [18]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.3 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
              appreciate = True           Republ : Democr =     14.0 : 1.0
                    drug = True           Republ : Democr =     10.3 : 1.0
                 special = True           Republ : Democr =     10.3 : 1.0
                   local = True           Republ : Democr =      9.9 : 1.0
                   elect = True           Democr : Republ =      9.6 : 1.0
                    land = True           Republ : Democr =      8.9 : 1.0
                  cities = True           Republ : Democr =      8.4 : 1.0
                citizens = True           Republ : Democr =      8.4 : 1.0
                    flag = True           Republ : Democr =      8.4 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Out of all the most informative feature words, 21 of the 25 words are
dominated by the Republican party. This is an interesting point because the
original dataset was dominantly speeches made by Democrats with 61% and only
39% for Republican speeches. This shows that speech patterns by Republicans
are very structured and use the same words. They are very aligned within their
messaging. In contrast, the Democratic speeches are not as
aligned and therefore the parties messaging and focus may not be as clear.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and
is unindexed, so the query takes a minute or two to run on my machine.


In [19]:
cong_db = sqlite3.connect(data_location / "congressional_data.db")
cong_cur = cong_db.cursor()

In [20]:
results = cong_cur.execute(
    """
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        """
)

results = list(results)  # Just to store it, since the query is time consuming

In [21]:
tweet_data = []

twitter_pipeline = [
    clean_tweet,
    tokenize_on_ws,
    partial(remove_punct, punct_set=TW_PUNCT_SET),
    is_alpha,
    lowercase,
    join_to_string,
]

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for result in results:
    cleaned = run_pipeline(result[2], twitter_pipeline)
    if cleaned:
        tweet_data.append((cleaned, result[1]))

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...


In [22]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data, k=10)

In [ ]:
def estimate_party(text): 
    t = text.split()
    

In [29]:
for tweet, party in tweet_data_sample:
    # you need to fill this in.
    # Fill in the right-hand side above with code that estimates the actual party
    features = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(features)
    # estimated_party = "??"

    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: rt auintard check out this video of my good friend sandra varner as she interviews congresswoman barbara lee httptcoqsncqeboto
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: going back amp forth between two committee meetings today making transport committee stop
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: trump will tell nato nations us cannot be the worlds piggy bank
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: rt tomperriello genuinely thankful to the people of our commonwealth for embracing an inclusive virginia and sending the most dynamic
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: walking around seeing people outside shopping having fun america is back in business nthank you realdonaldtrump
Actual party is Republican and our classifer says Republican.

Here's our (cleane

Now that we've looked at it some, let's score a bunch and see how we're doing.


In [32]:
# dictionary of counts by actual party and estimated party.
# first key is actual, second is estimated
parties = ["Republican", "Democratic"]
results = defaultdict(lambda: defaultdict(int))

for p in parties:
    for p1 in parties:
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data):
    tweet, party = tp
    # Now do the same thing as above, but we store the results rather
    # than printing them.
    features = conv_features(tweet, feature_words)
    # get the estimated party
    estimated_party = classifier.classify(features)
    # estimated_party = "Gotta fill this in"

    results[party][estimated_party] += 1

    if idx > num_to_score:
        break

In [33]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3056, 'Democratic': 1026}),
             'Democratic': defaultdict(int,
                         {'Republican': 4511, 'Democratic': 1409})})

### Reflections

_Write a little about what you see in the results_
